In [93]:
import numpy as np
import torch

In [94]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')

In [95]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [96]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2

In [97]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.4338, -1.1071, -1.0048],
        [ 0.3736, -0.3041, -1.2497]], requires_grad=True)
tensor([-1.0037, -2.5182], requires_grad=True)


In [98]:
def model(x):
    return x @ w.t() + b

In [99]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[-150.0567,  -49.3557],
        [-202.2159,  -75.2603],
        [-245.3778,  -83.2435],
        [-130.0392,  -23.7253],
        [-207.5568,  -93.4104]], grad_fn=<AddBackward0>)


In [100]:
# Compare with targets
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [101]:
# MSE loss
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [102]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(51588.3672, grad_fn=<DivBackward0>)


In [103]:
# Compute gradients
loss.backward()

In [104]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[-0.4338, -1.1071, -1.0048],
        [ 0.3736, -0.3041, -1.2497]], requires_grad=True)
tensor([[-21890.4121, -24781.3105, -15096.9180],
        [-12883.2266, -15097.3896,  -9214.1270]])


In [105]:
w
w.grad

tensor([[-21890.4121, -24781.3105, -15096.9180],
        [-12883.2266, -15097.3896,  -9214.1270]])

In [106]:
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5

In [107]:
# Let's verify that the loss is actually lower
loss = mse(preds, targets)
print(loss)

tensor(51588.3672, grad_fn=<DivBackward0>)


In [108]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [109]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[-110.9789,  -25.8720],
        [-150.8234,  -44.3522],
        [-184.3673,  -46.4588],
        [ -91.4665,   -0.6817],
        [-158.0919,  -63.5760]], grad_fn=<AddBackward0>)


In [110]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(35197.7461, grad_fn=<DivBackward0>)


In [111]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-17853.4531, -20436.6289, -12417.3574],
        [-10455.9824, -12481.9023,  -7601.6392]])
tensor([-215.3456, -128.1882])


In [112]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [113]:
print(w)
print(b)

tensor([[-0.0364, -0.6549, -0.7297],
        [ 0.6070, -0.0283, -1.0816]], requires_grad=True)
tensor([-0.9989, -2.5153], requires_grad=True)


In [114]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(24147.4785, grad_fn=<DivBackward0>)


In [115]:
# Train for 100 epochs
for i in range(10000):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [116]:
# Calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(0.4852, grad_fn=<DivBackward0>)


In [117]:
# Predictions
preds

tensor([[ 56.9988,  69.8890],
        [ 82.3109, 100.9328],
        [118.7207, 133.0285],
        [ 21.1027,  37.0687],
        [101.8931, 119.0687]], grad_fn=<AddBackward0>)

In [118]:
# Targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [119]:
import torch.nn as nn

In [120]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [121]:
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.],
        [ 74.,  66.,  43.],
        [ 91.,  87.,  65.],
        [ 88., 134.,  59.],
        [101.,  44.,  37.],
        [ 68.,  96.,  71.],
        [ 73.,  66.,  44.],
        [ 92.,  87.,  64.],
        [ 87., 135.,  57.],
        [103.,  43.,  36.],
        [ 68.,  97.,  70.]])

In [122]:
from torch.utils.data import TensorDataset

In [123]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [124]:
from torch.utils.data import DataLoader

In [125]:
# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [126]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[ 91.,  87.,  65.],
        [ 74.,  66.,  43.],
        [ 68.,  96.,  71.],
        [ 69.,  96.,  70.],
        [103.,  43.,  36.]])
tensor([[ 80., 102.],
        [ 57.,  69.],
        [104., 118.],
        [103., 119.],
        [ 20.,  38.]])


In [127]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.5433,  0.4024, -0.0778],
        [-0.1912,  0.1585,  0.1328]], requires_grad=True)
Parameter containing:
tensor([-0.4209,  0.1069], requires_grad=True)


In [128]:
# Parameters
list(model.parameters())

[Parameter containing:
 tensor([[-0.5433,  0.4024, -0.0778],
         [-0.1912,  0.1585,  0.1328]], requires_grad=True),
 Parameter containing:
 tensor([-0.4209,  0.1069], requires_grad=True)]

In [129]:
# Generate predictions
preds = model(inputs)
preds

tensor([[-16.4662,   2.4792],
        [-19.4287,   5.1545],
        [  1.7192,  12.4145],
        [-41.4111,  -7.6663],
        [ -4.7244,  11.4254],
        [-17.4118,   2.1295],
        [-19.9088,   5.1287],
        [  1.0982,  12.3560],
        [-40.4655,  -7.3166],
        [ -4.2589,  11.7493],
        [-16.9463,   2.4534],
        [-20.3743,   4.8048],
        [  2.1993,  12.4402],
        [-41.8766,  -7.9903],
        [ -3.7788,  11.7751]], grad_fn=<AddmmBackward0>)

In [130]:
# Import nn.functional
import torch.nn.functional as F

In [131]:
# Define loss function
loss_fn = F.mse_loss

In [132]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(8641.2881, grad_fn=<MseLossBackward0>)


In [133]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [134]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [135]:
fit(100, model, loss_fn, opt, train_dl)

Epoch [10/100], Loss: 89.4787
Epoch [20/100], Loss: 149.9156
Epoch [30/100], Loss: 96.0296
Epoch [40/100], Loss: 46.5229


Epoch [50/100], Loss: 35.1896
Epoch [60/100], Loss: 37.9778
Epoch [70/100], Loss: 16.8023
Epoch [80/100], Loss: 30.0410
Epoch [90/100], Loss: 21.9135
Epoch [100/100], Loss: 12.5168


In [136]:
# Generate predictions
preds = model(inputs)
preds

tensor([[ 57.4784,  71.2916],
        [ 79.6923,  98.5654],
        [122.8243, 135.7693],
        [ 23.8925,  42.6555],
        [ 95.9016, 112.2351],
        [ 56.2277,  70.2704],
        [ 79.0851,  98.2575],
        [122.8823, 136.1776],
        [ 25.1432,  43.6767],
        [ 96.5452, 112.9485],
        [ 56.8713,  70.9837],
        [ 78.4415,  97.5442],
        [123.4314, 136.0771],
        [ 23.2489,  41.9422],
        [ 97.1524, 113.2563]], grad_fn=<AddmmBackward0>)

In [137]:
# Compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [138]:
model(torch.tensor([[75, 63, 44.]]))

tensor([[53.4119, 68.0727]], grad_fn=<AddmmBackward0>)